In [7]:
try:
    import procgen_tools
except ImportError:
    get_ipython().run_line_magic(
        magic_name="pip",
        line="install -U git+https://github.com/ulissemini/procgen-tools",
    )

from procgen_tools.utils import setup

setup()  # create directory structure and download data

from procgen_tools.imports import *
from procgen_tools import maze, visualization
from typing import Tuple, Dict, List, Optional, Union

SAVE_DIR = "playground/visualizations"
AX_SIZE = 6

cheese_channels = [7, 8, 42, 44, 55, 77, 82, 88, 89, 99, 113]
effective_channels = [8, 55, 77, 82, 88, 89, 113]

Already downloaded https://nerdsniper.net/mats/model_rand_region_5.pth


In [ ]:
# def reaches_square(venv, hook, location: Tuple[int, int], max_steps: int = 70) -> bool:
#     """ Returns whether the agent reaches a given square in the maze
#     within `max_steps` time steps. Samples greedily from the hooked
#     network. """
#     assert (location[i] >= 0 and location[i] < maze.MAZE_SIZE for i in (0,1)).all()
#     assert max_steps >= 0

#     # At each time step, see if the agent is in the target square
#     for _ in range(max_steps):
#         # Get the grid position
#         grid_pos =

#         # Greedily sample from the hooked network

In [ ]:
def geometric_probability_path(
    start: Tuple[int, int], end: Tuple[int, int], vf: Dict
) -> float:
    for coord in (start, end):
        assert (coord[i] >= 0 and coord[i] < maze.MAZE_SIZE for i in (0, 1))

    # Get the path from start to end
    path = maze.pathfind(vf["grid"], start, end)

    print(path)


venv = maze.create_venv(num=1, start_level=0, num_levels=1)
vf = visualization.vector_field(venv, hook.network)
geometric_probability_path((0, 0), (0, 1), vf)